In [12]:
# --- Autoreload Commands ---
%load_ext autoreload
%autoreload 3

from typing import Any
from src import AgentTwo, FileTool_v2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
agent_tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of files in the directory.",
            "parameters": {
                "type": "object",
                "properties": {
                    "directory": {
                        "type": "string",
                        "description": "The directory path to list files from (e.g., './output', './', etc.)"
                    }
                },
                "required": ["directory"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {
                        "type": "string",
                        "description": "The path to the file to read relative to the current directory"
                    }
                },
                "required": ["file_name"]
            }
        }
    },
    {
    "type": "function",
    "function": {
        "name": "terminate",
        "description": "Terminates the conversation and provides the final response to the user.",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {
                    "type": "string",
                    "description": "The final message/summary to provide to the user"
                }
            },
            "required": ["message"]
        }
    }
}
]

In [72]:
def start_agentic_loop(agent: AgentTwo, file_tool: FileTool_v2, user_prompt: str) -> None:
  def should_continue(tool_call: Any) -> bool:
    return tool_call['function']['name'] != 'terminate'
  
  agent_response, tool_call = agent.user_prompt(user_prompt)
  print(f'Agent response: {agent_response}')
  print(f'Tool call: {tool_call}')
  
  while should_continue(tool_call):
    tool_response = file_tool.execute(tool_call)
    print(f'Tool response: {tool_response}')
    agent_response, tool_call = agent.tool_response(tool_response, tool_call['id'])
    print(f'Agent response: {agent_response}')
    print(f'Tool call: {tool_call}')
    
  # Execute the terminate tool to get the final message
  final_message = file_tool.execute(tool_call)
  print(f'\n=== Final Response ===\n{final_message}')
  

In [76]:
agent = AgentTwo(agent_tools)
file_tool = FileTool_v2()

start_agentic_loop(agent, file_tool, "What is in the sub-folder ./output of the current directory? I want a short summary of the files contents of any Python files.")

Memory:
system: 
You are an AI agent that can perform tasks by using available tools.

IMPORTANT: You have full access to the current working directory AND ALL of its subdirectories.
- You can list files in ANY subdirectory (e.g., './output', './src/utils', etc.)
- You can read files from ANY subdirectory using relative paths from the current directory
- Paths like './output/file.py' are completely valid and allowed

Workflow:
1. If a user asks about files, list them first before reading
2. When you want to read a file, provide its full relative path from the current directory
3. When you have gathered all necessary information, call the terminate tool with a comprehensive summary

Every response MUST call exactly one of the available tools.

user: What is in the sub-folder ./output of the current directory? I want a short summary of the files contents of any Python files.
--------------------------------
Response: ModelResponse(id='chatcmpl-4cade719-e608-4e69-b605-9fe51c103c24', creat